In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Pinecone
from langchain.text_splitter import NLTKTextSplitter, CharacterTextSplitter
import pinecone
from datetime import datetime


## Read PDF

In [11]:
pdf_path = "D:/Studium/BA-repo/dp-sent-analysis-twitter/thesis.pdf"
loader = PyPDFLoader(pdf_path)
document = loader.load()

## Split PDF into parts

In [12]:
# text in sätze splitten (chunk size anpassbar, also wie viele sätze (nach chars))
text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=0)

document_split = text_splitter.split_documents(document)

texts = [t.page_content for t in document_split]

print(texts[25])


Da diese Algorithmen nicht mit rohem Text arbeiten können, muss dieser in
numerische Werte oder Vektoren umgewandelt werden [15].

Dafür wird er in ein BoW-Modell
umgewandelt.

Ein BoW-Modell wird genutzt, um Features aus dem Rohtext zu gewinnen.

Um dieses Modell
generieren zu können, sind Informationen über das gegebene Vokabular, also die vorkommenden
Wörter, und über die Metrik, welche das Vorkommen dieser Wörter bewertet, notwendig.

Dabei
wird die Reihenfolge der Dokumente im Korpus vernachlässigt.

[15]
AufdereinenSeitesindeinzelneWörteralskleinstelinguistischeEinheitgeeignet,denFeatureraum
darzustellen.

Jedoch bleibt hierbei der Kontext, der durch die Aneinanderreihung dieser Wörter
entsteht, unbeachtet.

[8]
Für ein Dokument werden nun alle vorkommenden Wörter bestimmt, welche das Vokabular de-
finieren.

Mit jedem betrachteten Dokument wird dieses erweitert.

Ist das Vokabular vollständig,
können alle Dokumente in Wortvektoren umgewandelt werden.


## Create Embeddings

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from array import array
from pinecone import Vector

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV
)
index_name = "se4ai"

#specify embedding model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#upload embeddings into Pinecone vector store
docsearch = Pinecone.from_documents(document_split, embeddings, index_name=index_name)


## Similarity Search

In [14]:
query = "Welche Lernrate wurde verwendet?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

4.

Sentiment Analyse auf dem Datensatz
Abbildung 4.1.: „LRrangetest“[45]angewandtaufunserenDatensatz:KostenwertinAbhängigkeit
von der Lernrate.

angibt, welcher Anteil korrekt klassifiziert wurde, der Kostenwert jedoch zusätzlich quantifiziert,
wie sicher das Modell bei der Klassifizierung ist.

Zusätzlich wird Smiths Idee zum zyklischen Verändern der Lernrate angewandt [45].

Aller 4 Epo-
chen wird ein Zyklus durchlaufen, bei dem die Lernrate von 10−4auf10−2steigt, bevor sie wieder
auf10−4sinkt.

Der Vorteil liegt darin, dass Sattelpunkte der Kostenfunktion durch zeitweises Er-
höhen der Lernrate übersprungen werden können [45].

Eine zu kleine Lernrate könnte nicht den
nötigen Gradienten hervorbringen, um diesem Sattelpunkt zu entkommen.

Hier wird das Training
ebenfalls nach 7 Epochen ohne verbesserte Validierungskosten beendet.

In Abb.
